In [10]:
import numpy as np
import math
import matplotlib.pyplot as plt

In [29]:
import numpy as np
import math

# Discount factor
def discount_factor(r, T):
    return np.exp(-r * T)

# Forward rate
def forward_rate(T1, T2, r, tau=0.25):
    F = ((discount_factor(r, T1) / discount_factor(r, T2)) - 1) / tau
    return F

# Standard Normal PDF and CDF
def PDF(x):
    return (1 / math.sqrt(2 * math.pi)) * math.exp(-x**2 / 2)

def CDF(x):
    return 0.5 * (1 + math.erf(x / math.sqrt(2)))

# Black-76 Caplet/Floorlet Pricing
def Black76_option(F, K, sigma_ln, r, T, tau, notional, call=True):
    if F <= 0 or sigma_ln <= 0 or T <= 0:
        return 0.0

    vol_sqrtT = sigma_ln * math.sqrt(T)
    d1 = (math.log(F / K) + 0.5 * vol_sqrtT**2) / vol_sqrtT
    d2 = d1 - vol_sqrtT

    if call:
        price = discount_factor(r, T) * tau * (F * CDF(d1) - K * CDF(d2))
    else:
        price = discount_factor(r, T) * tau * (K * CDF(-d2) - F * CDF(-d1))

    return notional * price

# Bachelier Caplet/Floorlet Pricing
def Bachelier_option(F, K, sigma_ln, r, T, tau, notional, call=True):
    sigma_n = F * sigma_ln  # This mapping is exact only at ATM, for OTM/ITM it is approximately true. For deep OTM/ITM it is inaccurate
    if F <= 0 or sigma_n <= 0 or T <= 0:
        return 0.0

    denom = sigma_n * math.sqrt(T)
    d = (F - K) / denom

    if call:
        price = discount_factor(r, T) * tau * ((F - K) * CDF(d) + denom * PDF(d))
    else:
        price = discount_factor(r, T) * tau * ((K - F) * CDF(-d) + denom * PDF(d))

    return notional * price

# -----------------------------
# Example Inputs
# -----------------------------
T1, T2 = 1.0, 3.0
r = 0.035
tau = 0.25
notional = 100
K = 0.25
T = T1  # Option expiry
F = forward_rate(T1, T2, r, tau)
sigma_ln = 0.10 + 0.10 * math.exp(-T / 2.0)

# Price caplet and floorlet
caplet_black = Black76_option(F, K, sigma_ln, r, T, tau, notional, call=True)
floorlet_black = Black76_option(F, K, sigma_ln, r, T, tau, notional, call=False)

caplet_bach = Bachelier_option(F, K, sigma_ln, r, T, tau, notional, call=True)
floorlet_bach = Bachelier_option(F, K, sigma_ln, r, T, tau, notional, call=False)

# Output
print(f"Black-76 Caplet Price: {caplet_black:.6f}")
print(f"Black-76 Floorlet Price: {floorlet_black:.6f}")
print(f"Bachelier Caplet Price: {caplet_bach:.6f}")
print(f"Bachelier Floorlet Price: {floorlet_bach:.6f}")

Black-76 Caplet Price: 1.066438
Black-76 Floorlet Price: 0.100043
Bachelier Caplet Price: 1.088066
Bachelier Floorlet Price: 0.121671


### Portfolio of Caplet and Floorlet

In [30]:
import numpy as np
import math
import pandas as pd

# Discount factor
def discount_factor(r, T):
    return np.exp(-r * T)

# Forward rate
def forward_rate(T1, T2, r, tau=0.25):
    F = ((discount_factor(r, T1) / discount_factor(r, T2)) - 1) / tau
    return F

# Standard Normal PDF and CDF
def PDF(x):
    return (1 / math.sqrt(2 * math.pi)) * math.exp(-x**2 / 2)

def CDF(x):
    return 0.5 * (1 + math.erf(x / math.sqrt(2)))

# Volatility term structure
def sigma_ln_term(T):
    return 0.10 + 0.10 * math.exp(-T / 2.0)

# Black-76 caplet
def Black76_option(F, K, sigma_ln, r, T, tau, notional):
    vol_sqrtT = sigma_ln * math.sqrt(T)
    d1 = (math.log(F / K) + 0.5 * vol_sqrtT**2) / vol_sqrtT
    d2 = d1 - vol_sqrtT
    price = discount_factor(r, T) * tau * (F * CDF(d1) - K * CDF(d2))
    return notional * price

# Bachelier caplet
def Bachelier_option(F, K, sigma_ln, r, T, tau, notional):
    sigma_n = F * sigma_ln
    denom = sigma_n * math.sqrt(T)
    d = (F - K) / denom
    price = discount_factor(r, T) * tau * ((F - K) * CDF(d) + denom * PDF(d))
    return notional * price

# Portfolio of caplets
def price_cap_portfolio(T_start=0.25, T_end=5.0, tenor=0.25, r=0.035, notional=100, K=0.03):
    times = np.round(np.arange(T_start, T_end + tenor, tenor), 10)
    caplets = []
    total_black = 0.0
    total_bach = 0.0

    for i in range(len(times) - 1):
        T1, T2 = times[i], times[i + 1]
        T = T1
        tau = tenor
        F = forward_rate(T1, T2, r, tau)
        sigma_ln = sigma_ln_term(T)

        price_black = Black76_option(F, K, sigma_ln, r, T, tau, notional)
        price_bach = Bachelier_option(F, K, sigma_ln, r, T, tau, notional)

        caplets.append({
            'T1': T1, 'T2': T2, 'F': F, 'sigma_ln': sigma_ln,
            'Black76': price_black, 'Bachelier': price_bach
        })

        total_black += price_black
        total_bach += price_bach

    df = pd.DataFrame(caplets)
    return df, total_black, total_bach

# Run the portfolio pricing
df, total_black, total_bach = price_cap_portfolio(K=0.03)
print(df.head())
print(f"Total Cap Price (Black-76): {total_black:.4f}")
print(f"Total Cap Price (Bachelier): {total_bach:.4f}")

     T1    T2         F  sigma_ln   Black76  Bachelier
0  0.25  0.50  0.035154  0.188250  0.129155   0.129822
1  0.50  0.75  0.035154  0.177880  0.131565   0.133142
2  0.75  1.00  0.035154  0.168729  0.133656   0.135859
3  1.00  1.25  0.035154  0.160653  0.135146   0.137788
4  1.25  1.50  0.035154  0.153526  0.136122   0.139087
Total Cap Price (Black-76): 2.5675
Total Cap Price (Bachelier): 2.6328
